# Perceptor

The convergence of the perceptron is only guaranteed if the two classes are linearly separable and the learning rate is small enought.

If the classes can't be separated by a line, we can set a maximin number of passes over the training dataset (epochs) and/or a threshold for the number of tolerated missclassifactions.

![Linearly Separable and Not Linearly Separable ](https://github.com/edzzn/Machine_Learning/blob/master/images/linearly_separable_not_separable.png?raw=true)

# Perception Rule diagram
![](https://sebastianraschka.com/images/faq/classifier-history/perceptron-figure.png)

The perceptron recieves the inputs __x__ and combines them with the weights __w__.

Then the net input is passed to the activation funtion that outputs -1 or 1. In the learning phase the output is used to calculate the error and update the weights.

In [ ]:
# Implementation Perceptron Learning Algorithm

This is an implemetation on the __Iris Dataset__
